In [1]:
from src.vector_search import VectorSearch
import argparse
import logging
import os
from typing import List, Tuple
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from src.data.db import get_connection, release_connection
from src.config import DATA_DIR

load_dotenv()

2025-04-19 00:04:39.902 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/harshavardhan-patil/Work/Projects/canaria/canaria_ml_thp


/home/harshavardhan-patil/Work/Projects/canaria/canaria_ml_thp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Incorrect Duplicate example
With location filtering (including NULL)
Similarity Search on combined string of Job title, Company Name, Job Descriptions
Total 38k duplicates found at 0.824 threshold. Too permissive!

In [6]:
conn = get_connection()
cursor = conn.cursor()

cursor.execute("""
SELECT jobtitle_normalized, finalzipcode, jobdesc_clean from jobs_processed where lid IN ('a276e9c7623d84146f37f879babfbc98','e6fb2c5c1b9eb7a6ef5b29f6d7b2aa29')
""")

cursor.fetchall()

[('pt service associate',
  '19464',
  "Address: USA-PA-Stowe-180 Upland Square Drive \n Store Code: GC - Store Mgrs (2600291) \n \n At The GIANT Company we're committed to making our stores and facilities better every day for our team members, customers, and communities. It's our secret recipe for success and it guides everything we do. We put our customers first, do what's right, win together, and make ideas happen. As a team member here, that's exactly what you'll help us do. You'll share your ideas, learn new skills, and really make a difference for not only your team, but for your customers and your community. \n \n PRIMARY PURPOSE \n \n The primary purpose of this job is to bag items at checkout, put bags into carts, restock returns, retrieve carts from the parking lot, vacuum front-end, restock coolers, and collect trash. \n At The GIANT Company, we are passionate about building strong families and healthy communities, serving millions of neighbors across Pennsylvania, Maryland,